In [ ]:
# Imports for data processing
from sklearn.preprocessing import OneHotEncoder, MinMaxScaler, OrdinalEncoder
from sklearn.model_selection import train_test_split, KFold
from sklearn.compose import ColumnTransformer
from imblearn.over_sampling import SMOTE
from sklearn.impute import SimpleImputer
from imblearn.pipeline import Pipeline

# Imports for evaluation metrics
from sklearn.metrics import roc_auc_score, accuracy_score

# Imports for ML models
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import GaussianNB
from xgboost import XGBClassifier
from sklearn.svm import SVC

# Imports for hyperparameter optimization
import optuna as opt

# Other imports
from tqdm import tqdm
import pandas as pd
import numpy as np
import sys

# Ensure reproducibility
random_state = 0

## Import data and perform train and validation split

In [ ]:
# Import data
datapath = "data/healthcare-dataset-stroke-data-train.csv"
df = pd.read_csv(datapath)
df

In [ ]:
# Split the data into features and label
features = [col for col in list(df.columns) if col!="stroke"]
X, y = df[features], df["stroke"]

## Create the data preprocessor

Create the data preprocessing steps according to the EDA performed in file `1-eda.ipynb`:
- Impute missing values with the mean
- Perform one hot encoding on categorical columns

The EDA, however, misses one important step, which is the scaling of the data. It was not discussed in EDA because it actually makes it harder to understand the data. However, since we are using models that evaluate distances between datapoints, it is very important to scale our data.

Since the models we will be using do not assume normally distributed data, all features will be normalized in the [0, 1] range.

To summarize, the following steps are performed for preprocessing the data:
- Impute missing values with mean for the numerical columns
- One hot encoding for categorical columns
- MinMaxNormalization for every column

In [ ]:
# Create the mean imputer and feature scaler
num_features = [col for col in features if df[col].dtype in ["int64", "float64"]]
imputer = SimpleImputer()
scaler = MinMaxScaler()

# Create numerical preprocessor
num_preprocessor = Pipeline(steps=[
    ("imputer", imputer),
    ("scaler", scaler)
])

# Create the one-hot encoder
handle_unknown = "error"
one_hot_features = ["work_type", "smoking_status"]
one_hot_encoder = OneHotEncoder(handle_unknown=handle_unknown)

# Create ordinal encoder
cat_features = [col for col in features if col not in num_features]
ordinal_features = [col for col in cat_features if col not in one_hot_features]
ordinal_encoder = OrdinalEncoder()

# Create categorical preprocessor
cat_preprocessor = ColumnTransformer(transformers=[
    ("onehot", one_hot_encoder, one_hot_features),
    ("ordinal", ordinal_encoder, ordinal_features)
])

# Create the preprocessor
preprocessor = ColumnTransformer(transformers=[
    ("num_preprocessor", num_preprocessor, num_features),
    ("cat_preprocessor", cat_preprocessor, cat_features)
])

Finally, before going into the model selection phase, it is important to recall that the used dataset is imbalanced. More specifically, there are a lot fewer datapoints of patients who actually had a stroke. 

To compensate for their smaller frequency, we can increase their importance by giving them more weight. The following code cell calculates the class weights for the training data.

In [ ]:
def get_class_weights(X, y):
    # Create the class weights due to data imbalance
    largest_class_size = max([X[y==c].shape[0] for c in y.unique()])
    class_weights = {c: largest_class_size/X[y==c].shape[0] for c in y.unique()}
    return class_weights
    
get_class_weights(X, y)

However, some models are can't handle this type of argument to produce their results. For a more uniform approach across all model, an over-sampling technique, `SMOTE`, is applied using the `imblearn` package. This technique generates new datapoints for the minority class to balance the dataset.

In [ ]:
# Create dataset balancer
resampler = SMOTE(random_state=random_state)

In [ ]:
# Check that SMOTE balances the dataset
# NOTE: SMOTE here is applied to all data, later the data is split into training and validation, only training data should use SMOTE
X_res, y_res = resampler.fit_resample(preprocessor.fit(X, y).transform(X), y)
get_class_weights(X_res, y_res)

As we can see from the class weights, SMOTE does indeed balance our dataset!

## Model selection

To perform model selection, we will use cross validation to better evaluate our models on validation data. One thing to note is that we can not use SMOTE in our validation pipeline, only for training. This is because we want to train the model to give the same importance to each class, but to evaluate it in a dataset as close to the test set as possible (imbalaced). To do this, we will write a costum cross validation function:

In [ ]:
def cross_validate(X, y, pipe, n_splits, verbose=False):
    # Create folds for cross-validation
    kf = KFold(n_splits=n_splits)

    # Iterate folds for cross-validation
    train_score, val_score = 0, 0
    kfold_bar = enumerate(kf.split(X))
    if verbose:
        kfold_bar = tqdm(kfold_bar, total=n_splits, desc="Cross-validating", position=1, file=sys.stdout)
    for fold, (train_idx, val_idx) in kfold_bar:
        
        # Define training and validation sets
        X_train, X_val = X.loc[train_idx], X.loc[val_idx]
        y_train, y_val = y.loc[train_idx], y.loc[val_idx]
        
        # Fit pipeline to training data
        pipe = pipe.fit(X_train, y_train)
        
        # Get trained preprocessor and model
        preprocessor = pipe.get_params()["preprocessor"]
        model = pipe.get_params()["model"]
        
        # Save train data score
        y_train_pred = pipe.predict(X_train)
        train_score += roc_auc_score(y_train, y_train_pred) / n_splits
        
        # Define validation pipeline (remove SMOTE)
        val_pipe = Pipeline(steps=[
            ("preprocessor", preprocessor),
            ("model", model)
        ])
        
        # Save validation data score
        y_val_pred = val_pipe.predict(X_val)
        val_score += roc_auc_score(y_val, y_val_pred) / n_splits
        
    return train_score, val_score

To perform model selection, a few models were considered to solve this binary classification problem:
- Logistic regression classifier
- Support vector classifier
- Random forest classifier

They are all fitted to the training data according to the class weights and later evaluated on the validation dataset using the area under the ROC curve.

In [ ]:
# Create model pool to select from
model_pool = [
    {"name": "LR", "model": LogisticRegression(random_state=random_state), "train_score": 0, "val_score": 0},
    {"name": "SVM", "model": SVC(random_state=random_state), "train_score": 0, "val_score": 0},
    {"name": "RFC", "model": RandomForestClassifier(random_state=random_state), "train_score": 0, "val_score": 0},
    {"name": "KNN", "model": KNeighborsClassifier(), "train_score": 0, "val_score": 0},
    {"name": "DTC", "model": DecisionTreeClassifier(random_state=random_state), "train_score": 0, "val_score": 0},
    {"name": "GNB", "model": GaussianNB(), "train_score": 0, "val_score": 0},
    {"name": "XGB", "model": XGBClassifier(), "train_score": 0, "val_score": 0},
    {"name": "NN", "model": MLPClassifier(random_state=random_state, max_iter=500, early_stopping=True), "train_score": 0, "val_score": 0}
]

# Define the metric function and number of kfold splits
metric = roc_auc_score
n_splits = 5

# Iterate each model in the pool
model_bar = tqdm(model_pool, total=len(model_pool), desc="Iterating models", position=0, file=sys.stdout)
for model_dict in model_bar:
    
    # Get dictionary values
    name, model = model_dict["name"], model_dict["model"]
    
    # Define the training pipeline using the model
    train_pipe = Pipeline(steps=[
        ("preprocessor", preprocessor),
        ("resampler", resampler),
        ("model", model)
    ])
    
    # Cross validate
    train_score, val_score = cross_validate(X, y, train_pipe, n_splits, True)
    model_dict["train_score"], model_dict["val_score"] = train_score, val_score
        
    # Print final results
    tqdm.write(f"Model: {name} | Train score: {train_score:.3f} | Val score: {val_score:.3f}")

As we can see from these trained models, the one that faired better was the Logistic Regressor. However, both the Support Vector Machine classifier and the Neural Network also performed quite well. The Neural Network is especially promising, as it is very likely that performing hyperparameter tuning will boost its performance quite significantly.

## Hyperparameter optimization

Now that we know what the best models are, we will try to find their best hyperparameters so that we can get them to perform the best they can. We will be tuning:
- Logistic Regression Classifier
- Support Vector Machine Classifier
- Neural Networks Classifier

In order to do so, we will use an optimization package called `optuna`. First of all, let's define funtions to create the models and define their hyperparameter spaces:

In [ ]:
def LR_model(trial):
    solver = trial.suggest_categorical("solve", ["newton-cg", "lbfgs", "liblinear", "sag", "saga"]) # Solver hyperparameter
    c = trial.suggest_float("c", 1e-3, 1) # Hyperparameter for regularization strength
    model = LogisticRegression(solver=solver, C=c, random_state=random_state)
    return model
    
def SVM_model(trial):
    kernel = trial.suggest_categorical("kernel", ["linear", "poly", "rbf", "sigmoid"]) # Define kernel
    c = trial.suggest_float("c", 1e-3, 1) # Hyperparameter for regularization strength
    model = SVC(C=c, kernel=kernel, probability=True, random_state=random_state)
    return model

def NN_model(trial):
    num_hidden_layers = trial.suggest_int("num_hidden_layers", 1, 4) # Define number of hidden layers
    hidden_layer_sizes = [trial.suggest_int(f"hidden_layer_{i}", 5, 20) for i in range(num_hidden_layers)] # Define features per hidden layer
    model = MLPClassifier(hidden_layer_sizes=hidden_layer_sizes, random_state=random_state, max_iter=500, early_stopping=True)
    return model

Using these functions, we can now create an objective function, which we will use to optimize the ROC AUC metric for all of our models.

In [ ]:
def objective(trial, name):
    # Select model based on its name
    if name == "LR":
        model = LR_model(trial)
    elif name == "SVM":
        model = SVM_model(trial)
    else:
        model = NN_model(trial)
        
    # Define training pipeline on the model
    train_pipe = Pipeline(steps=[
        ("preprocessor", preprocessor),
        ("resampler", resampler),
        ("model", model)
    ])
    
    # Perform cross validation
    n_splits = 5
    _, val_score = cross_validate(X, y, train_pipe, n_splits)
    
    return val_score

Now we will create our model pool and perform hyperparameter optimization for each model:

In [ ]:
# Create model pool
model_pool = [
    {"name": "LR", "study": None},
    {"name": "SVM", "study": None},
    {"name": "NN", "study": None}
]

# Iterate each model in the pool
model_bar = tqdm(model_pool, total=len(model_pool), desc="Iterating models", position=0, file=sys.stdout)
for model_dict in model_bar:
    
    # Set tqdm bar postfix to model name
    name = model_dict["name"]
    model_bar.set_postfix(model=name)
    
    # Create optuna study
    sampler = opt.samplers.TPESampler(seed=random_state)
    study = opt.create_study(study_name=name, direction="maximize", sampler=sampler)
    
    # Optimize
    foo = lambda trial: objective(trial, name)
    study.optimize(foo, n_trials=20)
    model_dict["study"] = study